In [22]:
import json
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

brandFile = '/content/brand.csv'
receiptFile = '/content/receipt.csv'
userFile = '/content/user.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
brandDF = pd.read_csv(brandFile)
receiptDF = pd.read_csv(receiptFile)
userDF = pd.read_csv(userFile)

In [25]:
brandDF.shape

(1167, 8)

In [26]:
receiptDF.shape

(1119, 46)

In [27]:
userDF.shape

(495, 7)

In [24]:
## check for missing values

missing_values_brandDF = brandDF.isnull().sum()
missing_values_receiptDF = receiptDF.isnull().sum()
missing_values_userDF = userDF.isnull().sum()

In [4]:
## Here we can see that 612/1167 rows from the attribute 'topBrand' are null,
## which puts us in a dilemma of dropping this column or imputing the values.
## (since it's a boolean, it's better use randomforest imputation to prevent data loss)

missing_values_brandDF

barcode           0
category        155
categoryCode    650
name              0
topBrand        612
_id               0
cpg_ref           0
brandCode       269
dtype: int64

In [5]:
## ReceiptsDF has a lot of columns which have significant amout of missing / null
## values. We can clearly drop a good number of columns from this dataset.

missing_values_receiptDF

bonusPointsEarned                      575
bonusPointsEarnedReason                575
receiptPointsEarned                    510
purchasedItemCount                     484
rewardsReceiptStatus                     0
totalSpent                             435
userId                                   0
receipt_id                               0
createDate                               0
dateScanned                              0
finishedDate                           551
modifyDate                               0
pointsAwardedDate                      582
purchaseDate                           448
barcode                                558
description                            535
finalPrice                             454
itemPrice                              454
needsFetchReview                       985
partnerItemId                          440
preventTargetGapPoints                1002
quantityPurchased                      454
userFlaggedBarcode                    1002
userFlagged

In [6]:
## The user dataframe has relatively lesser nan values.
missing_values_userDF

active           0
role             0
signUpSource    48
state           56
userid           0
createDate       0
lastLogin       62
dtype: int64

In [7]:
# Check for duplicate values
duplicate_values_receiptDF = receiptDF.duplicated().sum()
duplicate_values_brandDF = brandDF.duplicated().sum()
duplicate_values_userDF = userDF.duplicated().sum()

In [8]:
duplicate_values_receiptDF

0

In [9]:
duplicate_values_brandDF

0

In [10]:
## This indicates that even though there are less nan values in user dataframe,
## we can see a lot of duplicates as well. However, presence of duplicates doesn't mean corrupt data.
duplicate_values_userDF

283

In [11]:
# Calculate descriptive statistics for each variable:

userDF.describe()

,active,role,signUpSource,state,userid,createDate,lastLogin
count,495,495,447,439,495,495,433
unique,2,2,2,8,212,212,172
top,True,consumer,Email,WI,54943462e4b07e684157a532,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204000000
freq,494,413,443,396,20,20,20


In [12]:
brandDF.describe()


,barcode
count,1.167000e+03
mean,5.111115e+11
std,2.874497e+05
min,5.111110e+11
25%,5.111112e+11
50%,5.111114e+11
75%,5.111117e+11
max,5.111119e+11


In [13]:
## The descriptive statistics on receipts suggest that certain columns are heavily
## skewed. Hence, it is better to either gather more data in order to balance the dataset.
## Or to eliminate skewed rows. So that machine learning models can be free of heavy biases.

receiptDF.describe()

,bonusPointsEarned,receiptPointsEarned,purchasedItemCount,totalSpent,finalPrice,itemPrice,partnerItemId,quantityPurchased,userFlaggedBarcode,userFlaggedPrice,userFlaggedQuantity,originalMetaBriteBarcode,discountedItemPrice,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,userFlaggedDescription,priceAfterCoupon
count,544.000000,609.000000,635.00000,684.000000,665.000000,665.000000,679.000000,665.000000,1.170000e+02,103.000000,103.000000,1.400000e+01,199.000000,3.800000e+01,6.0,169.000000,119.000000,0.0,100.000000
mean,238.893382,585.962890,14.75748,77.796857,11.328421,11.328421,105.409426,1.488722,6.786334e+09,24.524272,3.135922,6.588450e+10,18.562513,9.806868e+09,1.0,94.617751,763.546218,NaN,24.776100
std,299.091731,1357.166947,61.13424,347.110349,10.985227,10.985227,309.048998,1.193712,2.229414e+10,3.127502,1.379461,2.460178e+10,11.492729,2.045784e+10,0.0,166.094198,158.868816,NaN,5.541729
min,5.000000,0.000000,0.00000,0.000000,0.160000,0.160000,0.000000,1.000000,4.011000e+03,20.000000,1.000000,2.840064e+10,0.160000,4.023000e+03,1.0,5.000000,77.000000,NaN,0.790000
25%,5.000000,5.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,4.011000e+03,21.500000,2.000000,4.151999e+10,4.975000,4.023000e+03,1.0,5.000000,800.000000,NaN,22.970000
50%,45.000000,150.000000,2.00000,18.200000,9.990000,9.990000,1.000000,1.000000,4.011000e+03,25.000000,3.000000,8.087804e+10,22.970000,4.023000e+03,1.0,50.000000,800.000000,NaN,25.770000
75%,500.000000,750.000000,5.00000,34.960000,22.000000,22.000000,1.000000,1.000000,4.011000e+03,27.000000,4.000000,8.087804e+10,28.570000,1.356230e+10,1.0,100.000000,800.000000,NaN,28.570000
max,750.000000,10199.800000,689.00000,4721.950000,50.000000,50.000000,1235.000000,8.000000,7.940007e+10,29.000000,5.000000,8.087804e+10,50.000000,8.571830e+10,1.0,870.000000,800.000000,NaN,28.570000


# Major Data Issues and Remedies:



1.   Imbalanced Data: Receipts Data is skewed for certain attributes, which can be easily balanced out by row elimination, dummy data generation, or combining more data.

2.   Lots of Nans: Some columns which have a signifnicant nan values can be dropped off. Moreover, other nan values can be imputed with mean/mode/median or random forest imputing.

3.   Need of Feature Selection in Receipts: It's better to have consistent data, in order for a better predictive model. For that, we can easily remove highly correlated attributes.

4.   There are some inconsistencies in 'date' related attributes in Receipt Dataset. Which had to be converted to datetime format in order to avoid being used as numeric attributes, and instead being treated as a categorical attribute.

